# Day 11

Getting our input set up (we're also going to make a copy of the grid since it will be edited with each step)

In [62]:
import copy

In [63]:
fileText = open('day11Input.txt', 'r').read()
grid = fileText.split('\n')
for i in range(len(grid)): grid[i] = [int(i) for i in grid[i]]

gridMaster = copy.deepcopy(grid)

grid 

[[6, 1, 1, 1, 8, 2, 1, 7, 6, 7],
 [1, 7, 6, 3, 6, 1, 1, 6, 1, 5],
 [3, 5, 1, 2, 6, 8, 3, 1, 3, 1],
 [8, 5, 8, 2, 7, 7, 1, 4, 7, 3],
 [8, 2, 1, 4, 8, 1, 3, 8, 7, 4],
 [2, 3, 2, 5, 8, 2, 3, 2, 1, 7],
 [2, 2, 2, 2, 4, 8, 2, 8, 2, 3],
 [5, 4, 7, 1, 3, 5, 6, 7, 8, 2],
 [3, 7, 3, 8, 6, 7, 1, 2, 8, 7],
 [8, 6, 7, 5, 2, 2, 6, 5, 7, 4]]

## Part One

You enter a large cavern full of rare bioluminescent dumbo octopuses! They seem to not like the Christmas lights on your submarine, so you turn them off for now.

There are 100 octopuses arranged neatly in a 10 by 10 grid. Each octopus slowly gains energy over time and flashes brightly for a moment when its energy is full. Although your lights are off, maybe you could navigate through the cave without disturbing the octopuses if you could predict when the flashes of light will happen.

Each octopus has an energy level - your submarine can remotely measure the energy level of each octopus (your puzzle input). For example:
```
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
```
The energy level of each octopus is a value between 0 and 9. Here, the top-left octopus has an energy level of 5, the bottom-right one has an energy level of 6, and so on.

You can model the energy levels and flashes of light in steps. During a single step, the following occurs:

First, the energy level of each octopus increases by 1.
Then, any octopus with an energy level greater than 9 flashes. This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than 9, it also flashes. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)
Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.
Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy. Consider the middle octopus with 1 energy in this situation:

```
Before any steps:
11111
19991
19191
19991
11111
```
After step 1:
```
34543
40004
50005
40004
34543
```
After step 2:
```
45654
51115
61116
51115
45654
```
An octopus is highlighted when it flashed during the given step. Here is how the larger example above progresses:
```
Before any steps:
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526

After step 1:
6594254334
3856965822
6375667284
7252447257
7468496589
5278635756
3287952832
7993992245
5957959665
6394862637
```
After 100 steps, there have been a total of 1656 flashes.

Given the starting energy levels of the dumbo octopuses in your cavern, simulate 100 steps. How many total flashes are there after 100 steps?

**Step 1:** Find all the points that are next to any point

In [64]:
def isValidPoint(point):
    return point[0] in range(0,len(grid)) and point[1] in range(0, len(grid[0]))
    
def findNeighbors(point):
    i, j= point[0], point[1]
    neighbors = []
    pointsToCheck = [(i+1,j), (i-1,j), (i,j+1), (i,j-1), (i+1,j+1), (i-1,j+1), (i+1,j-1), (i-1,j-1)]
    for possibleNeighbor in pointsToCheck:
        if isValidPoint(possibleNeighbor): neighbors.append(possibleNeighbor)
    return neighbors

**Step 2:** Using the neighbors in order to make a graph of octopuses and the octopuses next to it

In [65]:
octopusGraph = {}
for i in range(len(grid)):
    for j in range(len(grid[0])): octopusGraph[(i, j)] = findNeighbors((i, j))

**Step 3:** Creating an increment function that will either increment the entire grid or just one octopus depending on the input

In [66]:
def increment(point=None):
    if point == None: #increment entire grid
        for i in range(len(grid)):
            for j in range(len(grid[0])): 
                grid[i][j] += 1
    else: grid[point[0]][point[1]] += 1 #increment one octopus

**Step 4:** Making a flash function that will make the octopus's energy level to zero and flash all the neighbors if they haven't been flashed already

In [67]:
def flash(point, flashed):
    grid[point[0]][point[1]] = 0
    if point not in flashed :
        flashed.add(point)
        for neighor in octopusGraph.get(point):
            if neighor not in flashed: increment(neighor)

**Step 4** Combining the previously made function to simulate an enitre step and return the total number of flashes per step

In [68]:
def step():
    increment()
    flashed = set()
    while True: #keep flashing until there's no more octopuses to flash
        flashCount = 0
        for i in range(len(grid)):
            for j in range(len(grid[0])): 
                if grid[i][j]  > 9: 
                    flash((i,j), flashed)
                    flashCount += 1
        if flashCount == 0: break #done flashing -> break
    return len(flashed)

**Step 5:** Simulating 100 steps

In [69]:
totalFlashes = 0
for i in range(100): totalFlashes += step()

Final Answer:

In [70]:
print(f'The total number of flashes after 100 steps is: {totalFlashes}')

The total number of flashes after 100 steps is: 1793


## Part Two
It seems like the individual flashes aren't bright enough to navigate. However, you might have a better option: the flashes seem to be synchronizing!

In the example above, the first time all octopuses flash simultaneously is step 195:
```
After step 193:
5877777777
8877777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777

After step 194:
6988888888
9988888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888

After step 195:
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
```
If you can calculate the exact moments when the octopuses will all flash simultaneously, you should be able to navigate through the cavern. What is the first step during which all octopuses flash?

**Step 1:** Resetting our grid and finding the octopus population

In [71]:
grid = copy.deepcopy(gridMaster)
octopusPopulation = len(octopusGraph)

**Step 2** Making steps until every octupus flashes in a step (when step's return == population)

In [72]:
allFlashStep, changed = 0, 0

while (changed != octopusPopulation):
    allFlashStep += 1
    changed = step()

Final Result:

In [73]:
print(f'The first time all the octopuses flash is step #{allFlashStep}')

The first time all the octopuses flash is step #247
